In [1]:
import numpy as np
import xarray as xr
import xrlint.api as xrl

In [2]:
xrl.import_rules()

In [3]:
nx = 2
ny = 3
nt = 4
ds = xr.Dataset(
    attrs=dict(title="SST-Cimatology Subset"),
    coords={
        "x": xr.DataArray(
            np.linspace(1000, 2000, nx),
            dims="x", 
            attrs={"units": "m"}
        ),
        "y": xr.DataArray(
            np.linspace(1000, 2000, ny),
            dims="y", 
            attrs={"units": "m"}
        ),
        "time": xr.DataArray(
            [2010 + y for y in range(nt)], 
            dims="time", 
            attrs={"units": "years"}),
    },
    data_vars={
        "sst": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin"}
        ),
        "sst_anomaly": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin"}
        )
    },
)

Pass the configuration of rules via `rules`. Pass a

- `dict[RuleSeveriry, RuleOptions]` to configure named rules,
- `list[str]` for multiple rule sets (not implemented yet),
- `str` to load a single prefined rule set (not implemented yet).

Here, `RuleSeverity` is either a

- `str`: one of `"error"`, `"warn"`, `"off"`
- `int`: one of `2` (error), `1` (warn), `0` (off)

and `RuleOptions` is a `dict[str, Any]`.

In [4]:
linter = xrl.Linter(rules={
    "no-empty-attrs": "warn",
    "dataset-title-attr": "error",
    "var-units-attr": "warn"
})

In [5]:
linter.verify_dataset(ds)

Result(file_path='<file>', messages=[], fixable_error_count=0, fixable_warning_count=0, error_count=0, fatal_error_count=0, warning_count=0)

In [6]:
invalid_ds = ds.copy()
invalid_ds.attrs = {}
invalid_ds.sst.attrs["units"] = 1

In [7]:
linter.verify_dataset(invalid_ds)

dataset.attrs,warn,Attributes are empty.,no-empty-attrs
dataset,error,Missing 'title' attribute in dataset.,dataset-title-attr
dataset.data_vars['sst'],warn,Invalid 'units' attribute in variable 'sst'.,var-units-attr
